In [1]:
# 한국어 위키피디아로 Word2Vec 학습하기

In [2]:
import os
import re

os.listdir('text')

['AC', 'AD', 'AJ', 'AE', 'AB', 'AG', 'AI', 'AH', 'AA', 'AF']

In [3]:
def list_wiki(dirname):
    filepaths = []
    filenames = os.listdir(dirname)
    for filename in filenames:
        filepath = os.path.join(dirname, filename)

        if os.path.isdir(filepath):
            filepaths.extend(list_wiki(filepath))
        else:
            find = re.findall(r"wiki_[0-9][0-9]", filepath)
            if 0 < len(find):
                filepaths.append(filepath)
    return sorted(filepaths)

In [4]:
filepaths = list_wiki('text')

In [5]:
with open("output_file.txt", "w") as outfile:
    for filename in filepaths:
        with open(filename) as infile:
            contents = infile.read()
            outfile.write(contents)

In [6]:
f = open('output_file.txt', encoding="utf8")

i = 0
while True:
    line = f.readline()
    if line != '\n':
        i = i+1
        print("%d번째 줄 :"%i + line)
    if i==10:
        break 
f.close()

1번째 줄 :<doc id="5" url="https://ko.wikipedia.org/wiki?curid=5" title="지미 카터">

2번째 줄 :지미 카터

3번째 줄 :제임스 얼 카터 주니어(, 1924년 10월 1일~)는 민주당 출신 미국의 제39대 대통령(1977년~1981년)이다.

4번째 줄 :생애.

5번째 줄 :어린 시절.

6번째 줄 :지미 카터는 조지아주 섬터 카운티 플레인스 마을에서 태어났다.

7번째 줄 :조지아 공과대학교를 졸업하였다. 그 후 해군에 들어가 전함·원자력·잠수함의 승무원으로 일하였다. 1953년 미국 해군 대위로 예편하였고 이후 땅콩·면화 등을 가꿔 많은 돈을 벌었다. 그의 별명이 "땅콩 농부" (Peanut Farmer)로 알려졌다.

8번째 줄 :정계 입문.

9번째 줄 :1962년 조지아주 상원 의원 선거에서 낙선하였으나, 그 선거가 부정선거 였음을 입증하게 되어 당선되고, 1966년 조지아 주지사 선거에 낙선하지만, 1970년 조지아 주지사 선거에서 당선됐다. 대통령이 되기 전 조지아주 상원의원을 두번 연임했으며, 1971년부터 1975년까지 조지아 지사로 근무했다. 조지아 주지사로 지내면서, 미국에 사는 흑인 등용법을 내세웠다.

10번째 줄 :대통령 재임.



In [28]:
from tqdm import tqdm
from konlpy.tag import Mecab 

In [8]:
mecab = Mecab()

In [9]:
f = open('output_file.txt', encoding="utf8")
lines = f.read().splitlines()
print(len(lines))

11129030


In [10]:
lines[:10]

['<doc id="5" url="https://ko.wikipedia.org/wiki?curid=5" title="지미 카터">',
 '지미 카터',
 '',
 '제임스 얼 카터 주니어(, 1924년 10월 1일~)는 민주당 출신 미국의 제39대 대통령(1977년~1981년)이다.',
 '생애.',
 '어린 시절.',
 '지미 카터는 조지아주 섬터 카운티 플레인스 마을에서 태어났다.',
 '조지아 공과대학교를 졸업하였다. 그 후 해군에 들어가 전함·원자력·잠수함의 승무원으로 일하였다. 1953년 미국 해군 대위로 예편하였고 이후 땅콩·면화 등을 가꿔 많은 돈을 벌었다. 그의 별명이 "땅콩 농부" (Peanut Farmer)로 알려졌다.',
 '정계 입문.',
 '1962년 조지아주 상원 의원 선거에서 낙선하였으나, 그 선거가 부정선거 였음을 입증하게 되어 당선되고, 1966년 조지아 주지사 선거에 낙선하지만, 1970년 조지아 주지사 선거에서 당선됐다. 대통령이 되기 전 조지아주 상원의원을 두번 연임했으며, 1971년부터 1975년까지 조지아 지사로 근무했다. 조지아 주지사로 지내면서, 미국에 사는 흑인 등용법을 내세웠다.']

In [11]:
result = []

for line in tqdm(lines):
  if line:
    result.append(mecab.morphs(line))

  0%|          | 1708/11129030 [00:00<25:36, 7243.71it/s] 

%(?:\((?P<key>.*?)\))?(?P<modifiers>[-#0-9 +*.hlL]*?)[eEfFgGdiouxXcrs%]


100%|██████████| 11129030/11129030 [04:51<00:00, 38156.00it/s] 


In [12]:
len(result)

7527200

In [13]:
from gensim.models import Word2Vec
model = Word2Vec(result, vector_size=100, window=5, min_count=5, workers=4, sg=0)

(?P<order1>[<>|=]?)(?P<repeats> *[(]?[ ,0-9]*[)]? *)(?P<order2>[<>|=]?)(?P<dtype>[A-Za-z0-9.?]*(?:\[[a-zA-Z0-9,.]+\])?)
\s*,\s*
\s+$
\*\*([0-9]*)
[a-z0-9_]+\(.*[,=].*\)
[0-9]+
numpy.dtype size changed
numpy.ufunc size changed
numpy.ndarray size changed
(\d+ | [a-z]+ | \.| -)
^\s*
    v?
    (?:
        (?:(?P<epoch>[0-9]+)!)?                           # epoch
        (?P<release>[0-9]+(?:\.[0-9]+)*)                  # release segment
        (?P<pre>                                          # pre-release
            [-_\.]?
            (?P<pre_l>(a|b|c|rc|alpha|beta|pre|preview))
            [-_\.]?
            (?P<pre_n>[0-9]+)?
        )?
        (?P<post>                                         # post release
            (?:-(?P<post_n1>[0-9]+))
            |
            (?:
                [-_\.]?
                (?P<post_l>post|rev|r)
                [-_\.]?
                (?P<post_n2>[0-9]+)?
            )
        )?
        (?P<dev>                                          # de

In [24]:
model_result1 = model.wv.most_similar("대한민국")
print(model_result1)

[('한국', 0.7307055592536926), ('미국', 0.6652010679244995), ('일본', 0.6484999656677246), ('부산', 0.5742846727371216), ('홍콩', 0.5724295377731323), ('서울', 0.561198353767395), ('태국', 0.5552341938018799), ('오스트레일리아', 0.5517556667327881), ('한경', 0.5381312370300293), ('제주특별자치도', 0.5345262289047241)]


In [15]:
model_result2 = model.wv.most_similar("어벤져스")
print(model_result2)

[('엑스맨', 0.7898859977722168), ('스파이더맨', 0.7788735628128052), ('퍼니셔', 0.7686418294906616), ('트와일라잇', 0.7561210989952087), ('어벤저스', 0.7561107873916626), ('아이언맨', 0.7497394680976868), ('인크레더블', 0.7463187575340271), ('데어데블', 0.741091251373291), ('배트맨', 0.7375123500823975), ('《》,', 0.7260107398033142)]


In [16]:
model_result3 = model.wv.most_similar("반도체")
print(model_result3)

[('집적회로', 0.8049557209014893), ('연료전지', 0.7972528338432312), ('웨이퍼', 0.7533517479896545), ('PCB', 0.7383823990821838), ('트랜지스터', 0.7358713150024414), ('윤활유', 0.7265740633010864), ('실리콘', 0.7221735715866089), ('그래핀', 0.7210060358047485), ('폴리머', 0.7206449508666992), ('전자', 0.7198912501335144)]
